In [67]:
import sys
print(sys.version)

3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]


In [68]:
import tobii_research as tr

In [69]:
import time
import csv
import os
import keyboard
from collections import defaultdict

Inicializo diccionario para contar los eventos (pulsar la tecla e)

In [70]:
event_count = defaultdict(int)
key_pressed = False

In [71]:
 global_user_position_guide = None

In [72]:
global_csv_filename = "gaze_data_1.csv"
event_csv_filename = "eventos_1.csv"

In [73]:
found_eyetrackers = tr.find_all_eyetrackers()

In [74]:
my_eyetracker = found_eyetrackers[0]

In [75]:
print("Address: " + my_eyetracker.address)
print("Model: " + my_eyetracker.model)
print("Name: " + my_eyetracker.device_name)
print("Serial number: " + my_eyetracker.serial_number)

Address: tobii-prp://TPNA1-030251340594
Model: Tobii Pro Nano
Name: 
Serial number: TPNA1-030251340594


Función que saca las **posiciones X e Y de cada ojo** y lo escribe en un csv:

Iba a plantearla sobreescribiendo el archivo de datos cada vez que se analice, pero la he dejado así por si queremos tener diferentes archivos con datos particulares, simplemente cambiar el nombre del csv a crear y así podremos crear distintos csvs con distintos datos con la misma función.

**IMPORTANTE: BORRAR EL CSV ANTES DE HACER QUE ESTA FUNCIÓN SE EJECUTE**

In [76]:
def gaze_data_callback(gaze_data):
    
#    print("Left eye: ({gaze_left_eye}, left pupil: {pupil_left_eye}) \t Right eye: ({gaze_right_eye}, Right pupil: {pupil_right_eye}, time: {timestamp})".format(
#    gaze_left_eye=gaze_data['left_gaze_point_on_display_area'],
#    gaze_right_eye=gaze_data['right_gaze_point_on_display_area'],
#    pupil_left_eye=gaze_data['left_pupil_diameter'],
#    pupil_right_eye=gaze_data['right_pupil_diameter'],
#    timestamp=gaze_data['device_time_stamp']))
    
    global event_count, key_pressed, global_csv_filename, event_csv_filename
    
    if keyboard.is_pressed('e') and not key_pressed:
        event_count['e']+=1
        key_pressed = True

        event_number = event_count['e']
        
        with open(event_csv_filename, mode='a', newline='') as file:
            event_writer = csv.writer(file)

            if file.tell() == 0:
                event_writer.writerow(['Evento','Timestamp'])
                
            event_writer.writerow([f'Evento {event_number}', gaze_data['system_time_stamp']])
                                      
    elif not keyboard.is_pressed('e'):
        key_pressed = False
    
    # Guardar los datos en un archivo CSV con nombres de columnas
    csv_filename = global_csv_filename

    # Definir nombres de columnas
    fieldnames = ['Left Eye X', 'Left Eye Y', 'Right Eye X', 'Right Eye Y', 'Pupil left eye', 'Pupil right eye', 'Timestamp device', 'Timestamp system']
#    fieldnames = ['Left Eye X', 'Left Eye Y', 'Right Eye X', 'Right Eye Y', 'Pupil left eye', 'Pupil right eye', 'Timestamp']
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Si el archivo está vacío, escribir encabezados
        if file.tell() == 0:
            writer.writeheader()

        # Obtener datos relevantes y formatearlos adecuadamente
        left_eye_data = gaze_data['left_gaze_point_on_display_area']
        right_eye_data = gaze_data['right_gaze_point_on_display_area']
        left_eye_pupil = gaze_data['left_pupil_diameter']
        right_eye_pupil = gaze_data['right_pupil_diameter']
        timestamp_device = gaze_data['device_time_stamp']
        timestamp_system = gaze_data['system_time_stamp']

        # Asegurarse de que los datos son del tipo adecuado para la escritura CSV
        left_eye_data = tuple(map(float, left_eye_data))
        right_eye_data = tuple(map(float, right_eye_data))
        

        # Crear un diccionario con nombres de columnas
        row_dict = {
            'Left Eye X': left_eye_data[0],
            'Left Eye Y': left_eye_data[1],
            'Right Eye X': right_eye_data[0],
            'Right Eye Y': right_eye_data[1],
            'Pupil left eye': left_eye_pupil,
            'Pupil right eye': right_eye_pupil,
            'Timestamp device': timestamp_device,
            'Timestamp system': timestamp_system
        }

        # Escribir datos en el archivo CSV
        writer.writerow(row_dict)

In [77]:
my_eyetracker.subscribe_to(tr.EYETRACKER_GAZE_DATA, gaze_data_callback, as_dictionary=True)


In [78]:
#time.sleep(5)


In [79]:
my_eyetracker.unsubscribe_from(tr.EYETRACKER_GAZE_DATA, gaze_data_callback)